<a href="https://colab.research.google.com/github/JayThibs/ai-safety-scrape/blob/main/scrape_ai_alignment_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Contents from AI Alignment Resources


# Installations

In [1]:
!pip install tika arxiv pandoc -q

     |████████████████████████████████| 81 kB 4.7 MB/s 
     |████████████████████████████████| 117 kB 47.7 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 


# Imports

In [4]:
import pandas as pd
from tika import parser
from google.colab import drive
from pathlib import Path
from urllib import request
from bs4 import BeautifulSoup

# Setting up Environment

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [9]:
CODE_DIR = Path('.') / 'code-projects/gpt-ai-safety'
RAW_DIR = Path('.') / 'data/raw'

In [10]:
%cd {CODE_DIR}

/content/drive/MyDrive/code-projects/gpt-ai-safety


In [11]:
# !git clone https://github.com/JayThibs/ai-safety-scrape
# !mv ai-safety-scrape/* .

# Load and Explore Data

In [12]:
df = pd.read_csv('ai-alignment-papers.csv')
df.head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,...,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,XBZAPQFK,blogPost,2020.0,"Kokotajlo, Daniel",Three kinds of competitiveness,AI Impacts,NaN,NaN,NaN,https://aiimpacts.org/three-kinds-of-competiti...,"By Daniel Kokotajlo In this post, I distinguis...",2020-03-30,2022-01-30 01:53:10,2022-01-30 01:53:10,2021-11-20 18:55:39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en-US,NaN,NaN,NaN,NaN,NaN,NaN,ZSCC: NoCitationData[s0] ACC: N/A Section: Blog,NaN,/Users/jacquesthibodeau/Zotero/storage/PU9A2KS...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,The international governance of artificial int...,2020,2022-01-30 04:47:43,2022-01-30 04:47:43,2021-11-13 15:58:24,545-556,NaN,5,11.0,NaN,NaN,Fragmentation and the Future,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,Wiley Online Library,NaN,ZSCC: 0000010 _eprint: https://onlinelibrary....,NaN,/Users/jacquesthibodeau/Zotero/storage/2TZBI3F...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BQCZM53S,blogPost,2021.0,"Clarke, Sam; Martin, Samuel Dylan",Distinguishing AI takeover scenarios,AI Alignment Forum,NaN,NaN,NaN,https://www.alignmentforum.org/posts/qYzqDtoQa...,Epistemic status: lots of this involves interp...,2021-09-08,2022-01-30 04:47:42,2022-01-30 04:47:42,2021-11-18 23:45:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZSCC: NoCitationData[s0] ACC: N/A,NaN,/Users/jacquesthibodeau/Zotero/storage/ENAMQXC...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,NaN,2020-12,2022-01-30 04:47:36,2022-01-30 04:47:36,2021-11-13 22:47:54,443-463,NaN,4,3.0,NaN,Int. Journal of Com. WB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000010,NaN,/Users/jacquesthibodeau/Zotero/storage/V3BEV7X...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,W8F6VI9I,thesis,2020.0,"Shah, Rohin Monish",Extracting and Using Preference Information fr...,NaN,NaN,NaN,NaN,https://www.proquest.com/openview/da8bf63ef343...,Typically when learning about what people want...,2020-12-17,2022-01-30 04:47:35,2022-01-30 04:47:35,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"University of California, Berkeley","Berkeley, CA",en,NaN,NaN,NaN,NaN,Zotero,NaN,ZSCC: 0000000,NaN,/

In [13]:
df['Item Type'].unique()

array(['blogPost', 'journalArticle', 'thesis', 'conferencePaper',
       'manuscript', 'report', 'bookSection', 'magazineArticle', 'book'],
      dtype=object)

In [14]:
item_nums = []
for item in df['Item Type'].unique():
    item_nums.append([item, len(df[df['Item Type'] == item])])

item_nums.sort(key=lambda x:x[1])
item_nums

[['magazineArticle', 2],
 ['thesis', 3],
 ['book', 13],
 ['bookSection', 52],
 ['report', 87],
 ['manuscript', 154],
 ['journalArticle', 170],
 ['conferencePaper', 262],
 ['blogPost', 421]]

In [15]:
df[df['Item Type'] == 'journalArticle'].head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,...,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,The international governance of artificial int...,2020,2022-01-30 04:47:43,2022-01-30 04:47:43,2021-11-13 15:58:24,545-556,NaN,5,11.0,NaN,NaN,Fragmentation and the Future,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,Wiley Online Library,NaN,ZSCC: 0000010 _eprint: https://onlinelibrary....,NaN,/Users/jacquesthibodeau/Zotero/storage/2TZBI3F...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,NaN,2020-12,2022-01-30 04:47:36,2022-01-30 04:47:36,2021-11-13 22:47:54,443-463,NaN,4,3.0,NaN,Int. Journal of Com. WB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000010,NaN,/Users/jacquesthibodeau/Zotero/storage/V3BEV7X...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,TK5F29IU,journalArticle,2021.0,"Hayden, Benjamin; Niv, Yael",The case against economic values in the orbito...,Behavioral Neuroscience,NaN,NaN,10.1037/bne0000448,https://osf.io/7hgup,Much of traditional neuroeconomics proceeds fr...,2021,2022-01-30 04:48:47,2022-01-30 04:48:47,2021-11-08 23:41:47,192-201,NaN,2,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOI.org (Crossref),NaN,ZSCC: 0000026 DOI: 10.31234/osf.io/7hgup,NaN,NaN,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NHWZIKZ2,journalArticle,2020.0,"Fernandes, Pedro; Santos, Francisco C.; Lopes,...",Norms for Beneficial A.I.: A Computational Ana...,AI Communications,NaN,"18758452, 09217126",10.3233/AIC-201502,http://arxiv.org/abs/1907.03843,The rise of artificial intelligence (A.I.) bas...,2020-12-18,2022-01-30 04:48:46,2022-01-30 04:48:46,2021-11-13 22:40:37,155-171,NaN,3-6,33.0,NaN,AIC,Norms for Beneficial A.I.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arXiv.org,NaN,ZSCC: 0000004 arXiv: 1907.03843,NaN,/Users/jacquesthibodeau/Zotero/storage/JAVXSVN...,NaN,UnsortedSafety,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,HDWGJGAP,journalArticle,2021.0,"Mingard, Chris; Valle-Pérez, Guillermo; Skalse...","Is SGD a Bayesian sampler? Well, almost",Journal of Machine Learning Research,NaN,NaN,NaN,http://arxiv.org/abs/2006.15191,Overparameterised deep neural networks (DNNs) ...,2021-02,2022-01-30 04:48:46,2022-01-30 04:48:46,2021-11-13 22:56:31,NaN,NaN,NaN,22.

In [ ]:
df_arxiv = df[df['Url'].str.contains('arxiv') == True]

In [ ]:
df_arxiv['Url'].str.replace('arxiv', 'ar5iv')

5       http://ar5iv.org/abs/2002.11328
7       http://ar5iv.org/abs/2010.11645
8       http://ar5iv.org/abs/2002.11708
9       http://ar5iv.org/abs/2012.10800
14      http://ar5iv.org/abs/2011.08512
                     ...               
1150    http://ar5iv.org/abs/1811.05590
1154    http://ar5iv.org/abs/1807.08364
1156    http://ar5iv.org/abs/1704.02882
1159    http://ar5iv.org/abs/1808.04096
1160    http://ar5iv.org/abs/1709.06166
Name: Url, Length: 317, dtype: object

In [1]:
def grab_text_from_webpage(url):
    with request.urlopen(url) as response:
        html = response.read()
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text(separator=" ")

In [2]:
arxiv_paper = grab_text_from_webpage('http://ar5iv.org/abs/2002.11328')

NameError: ignored

In [ ]:
arxiv_paper_list = arxiv_paper.split('\n\n')

In [ ]:
import os
import re
import pickle
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize


def cleanText(text):
    """Removes HTML tags and special characters from text."""

    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("\r", " ")
    text = text.replace("\xa0", " ")
    text = text.replace("\u200b", " ")
    text = text.replace("\u200c", " ")
    text = BeautifulSoup(text, "html.parser").get_text(
        separator=" "
    )  # remove html tags
    text = re.sub("\w{25,}", " ", text)  # remove long words
    text = re.sub("cid\d+", " ", text)  # remove cids
    text = re.sub(" s ", " ", text)  # remove s because of s tags
    text = re.sub(r"\s+", " ", text)  # remove extra spaces
    text = re.sub(r"", " ", text)  # remove bullet points
    text = re.sub(r"•", " ", text)  # remove numbers

    if multiplePeriods(text) is not None:
        return ""  # eliminates titles from paragraph list
    elif tableTitle(text) is not None:
        return ""
    elif figureTitle(text) is not None:
        return ""
    elif photoTitle(text) is not None:
        return ""

    # remove whitespace at the beginning and end of the text
    if len(text) > 25:
        if text[0] == " ":
            text = text[1:]
        if text[-1] == " ":
            text = text[:-1]
        return text
    else:
        return ""

In [ ]:
list_text = BeautifulSoup(arxiv_paper, "html.parser").get_text(separator=" ").split('\n')

In [ ]:
''.join(list_text)

'[2002.11328] Rethinking Bias-Variance Trade-off for Generalization of Neural Networks  function detectColorScheme(){    var theme="light";    var current_theme = localStorage.getItem("ar5iv_theme");    if(current_theme){      if(current_theme == "dark"){        theme = "dark";      } }    else if(!window.matchMedia) { return false; }    else if(window.matchMedia("(prefers-color-scheme: dark)").matches) {      theme = "dark"; }    if (theme=="dark") {      document.documentElement.setAttribute("data-theme", "dark");    } else {      document.documentElement.setAttribute("data-theme", "light"); } }    detectColorScheme();    function toggleColorScheme(){    var current_theme = localStorage.getItem("ar5iv_theme");    if (current_theme) {      if (current_theme == "light") {        localStorage.setItem("ar5iv_theme", "dark"); }      else {        localStorage.setItem("ar5iv_theme", "light"); } }    else {        localStorage.setItem("ar5iv_theme", "dark"); }    detectColorScheme(); }Rethi

# Extract text from AI Alignment Resources

In [16]:
def tikaTextExtractor(file_path):
    """Extracts text from a PDF using tika."""
    print("Extracting text from file: " + file_path)
    parsed_tika = parser.from_file(file_path)
    return parsed_tika["content"]

In [18]:
txt = tikaTextExtractor(str(RAW_DIR / 'pdfs' / 'Superintelligence.pdf'))

Extracting text from file: data/raw/pdfs/Superintelligence.pdf


In [22]:
with open(RAW_DIR / 'txts' / 'Superintelligence.txt', 'w') as f:
    f.write(txt)

In [23]:
with request.urlopen('https://arxiv.org/e-print/2010.11645') as response:
    html = response.read()

In [24]:
html

b'\x1f\x8b\x08\x00\x91\x03\xa2_\x00\x03\xec\\k\x93\xdb6\x96\xcdg\xfd\n|P\xb2\xdd\xb1\xa4\x91\xec\xd8I\x9c\xea\x9ar\xec8\xf6\xae\x1f=\xee\xcedwLw\x17EB\x12\xdc\x14I\x13\xa4\xbae\x85\xfe\xed{\xee\x05@\x82z\xd8\xde\xdaJRS\x15\xa5\xc6-\x91 pqq\x1f\xe7>8\xe1T\x97E\x18\x95\xa3R\xde|\xf1;}\xc6\xf8\xdc\xbb\xf7\r\xff\xc5g\xfb\xef\xed\xbbwo\x7f1\xb9\xf3\xed\xdd\xf1\x1d\xfc;\xb9\xf7\xc5xr{2\xb9\xfb\x85\x18\xff^\x04\xf9\x9fJ\x97a!\xc4\x17E\x96\x95\x1f\x1b\xf7\xa9\xfb\xff\xa6\x9f`*\xe7*\xdd\x84V\x0c\xea\xde\xc3,]\xc9\x1bQ\xc8$\xbc\tK\x95\xa5Z,\xc2\x95\x14r)\x8b\xb9\x8cE\xa8E(\xf2"[*\xad\xd2\xb9\x08s|\x0f\xa3\x85\x98e\x85X\xc9B\xcd\xd6t9\x96Z\x15\xe14\x9144\x97E\xa9\xa4\x16\xd9L\xa4\xb2*\xc2\x04\x7f\xca\xeb\xac\xb8\xd2"QWR\x14\xd9\x14\xa7\x90J\xadE\x99\x890\xc64:,\x14\xc6\xd1\x93U15t\x8c\xc4\xaf*\x96\xc9ZT\x1at<S\xa9\xc4\xc1\x9d\x16\xd9\xbc\x08\x97KZ\xf4\xe8\xd9\xe9q\x87\xf0,\xc5hZH\\\xcb$\x11\xd7\x0b\x99\xb6K\x87\x85\x14\xd83\xa6\x89i\xd9Y\x18\xa9D\x95a)\xcd6T\xc4\x93\x8c\xc4\xf9Bi\xecBFI\x85]\xd1\

In [ ]:
import requests

for i,link in enumerate(['https://arxiv.org/e-print/2010.11645']):
    r = requests.get(link,allow_redirects=True)
    with open(f"a{i}.","wb") as code:
        code.write(r.content)

In [27]:
import pandoc

In [30]:
pandoc.read(file='/content/drive/MyDrive/code-projects/gpt-ai-safety/2006.15191')

ProcessExecutionError: ignored

     |████████████████████████████████| 81 kB 3.9 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=caa5b1ad127442ce995d94b23a1979a0c1760c8fb66b17e3538b45c155c9d2d0
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [39]:
import arxiv
paper = next(arxiv.Search(id_list=["2101.00027v1"]).results())

In [40]:
paper.download_source(dirpath="./data/raw/latex", filename="2101.00027v1.tar.gz")

'./data/raw/latex/2101.00027v1.tar.gz'

In [51]:
paper.entry_id

'http://arxiv.org/abs/2101.00027v1'

In [50]:
tar = tarfile.open("data/raw/latex/2101.00027v1.tar.gz")
for member in tar.getmembers():
    if ".tex" in member.name:
        tar.extract(member, LATEX_DIR)
tar.close()

In [49]:
import os
os.listdir()

['ai-safety-scrape',
 'README.md',
 'main.py',
 'scrape_ai_alignment_content.ipynb',
 'data',
 'process.py']

In [ ]:
def download_arxiv_paper_tar(paper_id, tar_filename):
    paper = next(arxiv.Search(id_list=[paper_id]).results())
    paper.download_source(dirpath=TAR_DIR, filename=filename)

def extract_tex(tar_filename):
    tar = tarfile.open(str(TAR_DIR / tar_filename))
    for member in tar.getmembers():
        if ".tex" in member.name:
            tar.extract(member, LATEX_DIR)
    tar.close()

In [ ]:
for paper_id in arxiv_paper_list:
    tar_filename = paper_id + ".tar.gz"
    extract_tex(tar_filename)